In [1]:
import sys
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
import time
from IPython.core.display import display, HTML                                # Make the Cell width of the notebook wider
display(HTML("<style>.container { width:80% !important; }</style>"))        

In [11]:
def INT_REES(phi_nm1 , phi_n,  R_n , S_n , y_nm1 , y_n , y_np1):      

    dy      = y_np1 - y_n 
    denom   = 1.0 / (dy*dy) + (R_n/(y_np1 - y_nm1))               
    num2    =  S_n * phi_n - (R_n * phi_nm1 / (y_np1 - y_nm1))
    num1    =  (-2.0 * phi_n + phi_nm1) / (dy*dy) 
    phi_np1 = - (num1 + num2) / denom

    return phi_np1

In [13]:
def R_S_Rees(y, c_n , k):   
    
    len_y = len(y)  
  
    # Allocate memory for the numpy arrays   
    R      = np.zeros( len_y, dtype =  np.complex )     
    S      = np.zeros( len_y, dtype =  np.complex )
    Rv     = np.zeros( len_y, dtype =  np.complex )
    Sv     = np.zeros( len_y, dtype =  np.complex )
    Rs     = np.zeros( len_y, dtype =  np.complex )
    Ss     = np.zeros( len_y, dtype =  np.complex )
    f      = np.zeros( len_y, dtype =  np.complex )
    B      = np.zeros( len_y, dtype =  np.complex )
    H      = np.zeros( len_y, dtype =  np.complex )
    dHdy   = np.zeros( len_y, dtype =  np.complex )
    U      = np.zeros( len_y, dtype =  np.complex )
    dUdy   = np.zeros( len_y, dtype =  np.complex )
    HQ     = np.zeros( len_y, dtype =  np.complex )
    dHQdy  = np.zeros( len_y, dtype =  np.complex )
    D      = np.zeros( len_y, dtype =  np.complex )
    dDdy   = np.zeros( len_y, dtype =  np.complex )
    F      = np.zeros( len_y, dtype =  np.complex )
    dFdy   = np.zeros( len_y, dtype =  np.complex )
    G      = np.zeros( len_y, dtype =  np.complex )
    dGdy   = np.zeros( len_y, dtype =  np.complex )
    d2Udy2 = np.zeros( len_y, dtype =  np.complex )

    # Define the constants
    beta   = 2.2*(10**-11)       
    g      = 0.0334              # Reduced gravity
    H0     = 140                  # Average thermocline depth (see Tanaka+Hibiya)
    A      = [-0.5,-0.5,-5,-0.5]
    c      = np.sqrt(g*H0)      
    C      = np.sqrt((2*beta/c))  
    #y     = C*x                 # Non-dimensional y
    i      = 5                   # Choose your profile: i = [0 (n=0), 1 (n=2), 2 (Cubic), 3 (Quintic) , 4 (Sinusoidal), 5(Cosine)]
    L      = 10

    for iy in range ( len_y ) :
        if i == 0:
            f[iy]     = beta*y[iy]           
            B[iy]     = (y[iy]**2)/4.0  
            H[iy]     = A[0] * np.exp(-B[iy])                                                     
            dHdy[iy]  = - (A[0] / 2.0) * C * y[iy] * np.exp(-B[iy])                               
            U[iy]     = (g * A[0] / c) * np.exp(-B[iy])                                           
            dUdy[iy]  = -(g * A[0] / (2*c))*C*y[iy]*np.exp(-B[iy])
            HQ[iy]    = beta * (1 / C) * y[iy] + (g * A[0] / 2 * c) * C * y[iy] * np.exp(-B[iy])  
            dHQdy[iy] = beta - (g*A[0]/4*c)*(C**3)*((y[iy]**2)-2)*np.exp(-B[iy])
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))                                                         
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))        
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
            
        if i == 1:
            f[iy]     = beta*y[iy]
            B[iy]     = (y[iy]**2)/(4.0)
            H[iy]     = A[2]*((y[iy]**2)-1)*np.exp(-B[iy])
            dHdy[iy]  = -(A[2]/2.0)*C*y[iy]*((y[iy]**2)-5)*np.exp(-B[iy])
            U[iy]     = -(g*A[2]/c)*((y[iy]**2)-5)*np.exp(-B[iy])
            dUdy[iy]  = (g*A[2]/(2*c))*C*y[iy]*np.exp(-B[iy])*((y[iy]**2)-9) 
            HQ[iy]    = beta*(1/C)*y[iy] + (g*A[2]/(2*c))*C*y[iy]*np.exp(-B[iy])*((y[iy]**2)+9) 
            dHQdy[iy] = beta + (g*A[2]/2*c)*C*(2*C*(y[iy]**2)*np.exp(-B[iy])+((y[iy]**2)+9)*(-0.5*C*((y[iy]**2)-2)*np.exp(-B[iy])))
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))  
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
            
        if i == 2:
            f[iy]     = beta*y[iy]
            B[iy]     = (y[iy]**2)/4.0
            H[iy]     = (y[iy]**3)*np.exp(-B[iy])
            dHdy[iy]  = 3*(y[iy]**2)*C*np.exp(-B[iy])-0.5*C*(y[iy]**4)*np.exp(-B[iy])
            U[iy]     = (g/c)*y[iy]*np.exp(-B[iy])*((y[iy]**2)-6)
            dUdy[iy]  =  (g/c)*(-0.5*C*((y[iy]**2)-2)*((y[iy]**2)-6)*np.exp(-B[iy])+2*C*(y[iy]**2)*np.exp(-B[iy]))
            HQ[iy]    = beta*(1/C)*y[iy] - (g/c)*(-0.5*C*((y[iy]**2)-2)*((y[iy]**2)-6)*np.exp(-B[iy])+2*C*(y[iy]**2)*np.exp(-B[iy]))
            dHQdy[iy] = beta - (g/c)*C*((-0.5*C*y[iy]*np.exp(-B[iy]))*(-0.5*((y[iy]**2) -2)*((y[iy]**2) -6)+2*(y[iy]**2)) + np.exp(-B[iy])*(-2*C*(y[iy]**3)+12*y[iy]*C))
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))  
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))  
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
            
        if i == 3:
            f[iy]     = beta*y[iy]
            B[iy]     = (y[iy]**2)/4.0
            H[iy]     = (y[iy]**5)*np.exp(-B[iy])
            dHdy[iy]  = -0.5*C*(y[iy]**4)*np.exp(-B[iy])*((y[iy]**2)-10)
            U[iy]     = (g/c)*(y[iy]**3)*np.exp(-B[iy])*((y[iy]**2)-10)
            dUdy[iy]  = (g/c)*(-(1/2)*C*((y[iy]**2))*((y[iy]**2)-6)*((y[iy]**2)-10)*np.exp(-B[iy])+2*C*(y[iy]**4)*np.exp(-B[iy]))
            HQ[iy]    = beta*(1/C)*y[iy] - (g/c)*(-(1/2)*C*((y[iy]**2))*((y[iy]**2)-6)*((y[iy]**2)-10)*np.exp(-B[iy])+2*C*(y[iy]**4)*np.exp(-B[iy]))
            dHQdy[iy] = beta - (g/c)*C*(-0.5*C*y[iy]*np.exp(-B[iy])*(-0.5*(y[iy]**2)*((y[iy]**2) -6)*((y[iy]**2) -10)+2*(y[iy]**4))+np.exp(-B[iy])*(6*C*(y[iy]**5)-56*C*(y[iy]**3)+120*C*y[iy]))
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
            
        if i == 4:
            f[iy]     = beta*y[iy]
            B[iy]     = (y[iy]**2)/(4.0)
            m         = 1
            U[iy]     = A[0]*np.sin(m*np.pi*y[iy] / (C*L))   
            H[iy]     = -(A[0]*(beta) / g)*((L/m*np.pi)**2)*((np.sin(m*np.pi*y[iy]/(C*L))-(L/(m*np.pi*C))*y[iy]*np.cos(m*np.pi*y[iy]/(C*L))))
            dUdy[iy]  = A[0]*(m*np.pi/L)*np.cos(m*np.pi*y[iy]/(C*L))
            dHdy[iy]  = -(beta/(C*g))*y[iy]*A[0]*np.sin(m*np.pi*y[iy]/(C*L))
            HQ[iy]    = (beta/C)*y[iy] - A[0]*(m*np.pi/L)*np.cos(m*np.pi*y[iy]/(C*L))
            dHQdy[iy] =  beta + A[0]*((m*np.pi/L)**2)*np.sin(m*np.pi*y[iy]/(C*L)) 
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v 
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
            
        if i == 5:
            f[iy]     = beta*y[iy]
            B[iy]     = (y[iy]**2)/(4.0)
            m         = 1
            U[iy]     = A[0]*np.cos(m*np.pi*y[iy]/(L))   
            H[iy]     = -(A[0]*(beta) / g)*((L/m*np.pi)**2)*((np.cos(m*np.pi*y[iy]/(L))+(L/(m*np.pi))*y[iy]*np.sin(m*np.pi*y[iy]/(L))))
            dUdy[iy]  = -A[0]*(m*np.pi/L)*np.sin(m*np.pi*y[iy]/(L))
            dHdy[iy]  = -(beta/(g))*y[iy]*A[0]*np.cos(m*np.pi*y[iy]/(L))
            HQ[iy]    = (beta)*y[iy] + A[0]*(m*np.pi/L)*np.sin(m*np.pi*y[iy]/(L))
            dHQdy[iy] =  beta + A[0]*((m*np.pi/L)**2)*np.cos(m*np.pi*y[iy]/(L)) 
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
        
 
    return R, S, U, H, dUdy, dHdy, HQ, dHQdy, D ,dDdy, F, dFdy, G, dGdy, f, Rv, Sv, Rs, Ss

In [14]:
start_time=time.time()

Levels      = 100                 # Number of levels
Tolerance   = 1e-7                # Tolerance for errors
beta        = 2.2*(10**-11)       
g           = 0.0334              # Reduced gravity
H0          = 140                  # Average thermocline depth (see Tanaka + Hibiya 2019)
c           = np.sqrt(g*H0) 
C           = np.sqrt((2*beta/c)) 

SMALL_SIZE  = 14                # Set the sizes for the text in plots
MEDIUM_SIZE = 16
BIGGER_SIZE = 22

y  = np.zeros(Levels + 2)       # Allocate memory to y
dy = 1.0 / float(Levels)        # float/float = float
L = 10                          # Length of half the domain
M = 2*L                         
for n in range(Levels + 2):
    y[n] = -L + M * n * dy  
    

wavespeedre = [0.49]   # -0.49776863
wavespeedim = [0.02]   #  0.005120403
wavenumber  = [1e-15 + 0.01] 
wsincrement = [0.01]
wnincrement = [0.01]

c_0_re  = wavespeedre[0]
c_0_im  = wavespeedim[0]                 # Imaginary part of the prescribed wave speed
c_0     = c_0_re + 1j * c_0_im                  
d_c_inc = wsincrement[0]                         # Increment that we will add to the previous wave speed after each loop
d_c     = d_c_inc + 1j * d_c_inc                 # Since c is complex we need to make the increment complex too
c_1     = c_0  + d_c                             # This is the next step in wave speed which we use to calculate c_{new}
k       = wavenumber[0]                          # Prescribed choice of wavenumber (can't have k=0 so start near to it instead)
k_final = 0.28                                   # Wavenumber to iterate up to
d_k     = wnincrement[0]                         # Similarly, we increment over the wavenumber after each loop


Max_iter_1 = 1 #int(k_final/d_k) 

K       = np.zeros(Max_iter_1)                      
cs      = np.zeros(len(K),dtype=np.complex) 

for iter_outer in range(Max_iter_1):
    print(f'{iter_outer}')
    K[iter_outer] = k 
    PHIBC0 = np.zeros(Max_iter_1)

    R, S, U, H, dUdy, dHdy, HQ, dHQdy, D, dDdy, F, dFdy, G, dGdy, f, Rv, Sv, Rs, Ss = R_S_Rees(y, c_0 , k)  

    PHI = np.zeros(Levels + 2, dtype = np.complex) # Allocate memory for phi
    PHI[0] = 0.0                                   # Boundary condition
    PHI[1] = 1.0                                   # Sets the otherwise arbitrary amplitude

    #Integrate up through the domain
    for n in range(1,Levels + 1): # PHI[2] to PHI[Levels + 1]
        PHI[n+1] = INT_REES(PHI[n-1],PHI[n],Rs[n],Ss[n],y[n-1],y[n],y[n+1])

    PHI_NU_BC_0 = PHI[Levels]  # Use this later (see Section 3.9) to determine convergence to the RHS boundary condition

    #Start iterating towards a solution (i.e. increment c forwards)
    Max_iter_2 = 50      # Number of iterations

    Iter_Inner=np.linspace(1,Max_iter_2,Max_iter_2)

    for iter_inner in range(Max_iter_2):

        R, S, U, H, dUdy, dHdy, HQ, dHQdy, D, dDdy, F, dFdy, G, dGdy, f, Rv, Sv, Rs, Ss = R_S_Rees(y,c_1,k) 

        PHI = np.zeros(Levels + 2, dtype = np.complex) # Allocate memory for phi
        PHI[0] = 0.0                                   # Boundary condition
        PHI[1] = 1.0                                   # Normalise the amplitude

        #Integrate up through the domain
        for n in range(1,Levels + 1):
            PHI[n+1] = INT_REES(PHI[n-1],PHI[n],Rs[n],Ss[n],y[n-1],y[n],y[n+1]) # For c_1, find all the phi in the domain

        PHI_NU_BC_1 = PHI[Levels]    # Use this later (see Section 3.9) to determine convergence to the RHS boundary condition 


        if abs(PHI_NU_BC_1) < Tolerance:   # If PHI[Levels] is close to zero then we save the c that satifies this      
            cs[iter_outer] = c_1                     
            c_0 = c_1

        if abs(PHI_NU_BC_1) > Tolerance and iter_inner < Max_iter_2:   # if PHI at the RHS boundary is not close to zero then
            gradient = (c_1 - c_0) / (PHI_NU_BC_1 - PHI_NU_BC_0)
            c_new = c_1 - gradient * PHI_NU_BC_1

            with open('A_Shooting Method Diagnostic.txt','a') as file_out:
                print(f'Wavenumber =  {K[iter_outer]:.3f}',file=file_out)
                print(f'',file=file_out)
                print(f'iter_inner = ',iter_inner             ,file=file_out)
                print(f''                                     ,file=file_out)
                print(f'c_0 = ',c_0                 ,file=file_out)
                print(f''                                     ,file=file_out)
                print(f'c_1 = ',c_1                 ,file=file_out)
                print(f''                                     ,file=file_out)
                print(f'PHI_NU_BC_0 = ',PHI_NU_BC_0           ,file=file_out)
                print(f''                                     ,file=file_out)
                print(f'PHI_NU_BC_1 = ',PHI_NU_BC_1           ,file=file_out)
                print(f''                                     ,file=file_out)
                print(f'gradient = ',gradient       ,file=file_out)
                print(f''                                     ,file=file_out)
                print(f'c_new = ',c_new             ,file=file_out)
                print(f''                                     ,file=file_out)

            if abs(PHI_NU_BC_1) < abs(PHI_NU_BC_0): # if PHI is converging to zero then update the wave speeds
                c_0 = c_1
                PHI_NU_BC_0 = PHI_NU_BC_1

            c_1 = c_new

        if iter_inner == Max_iter_2 - 1:
            #print(f'Solution failed to converge')
            break


    K[iter_outer] = k   # K stores the wavenumbers that we iterate over
    k = k + d_k         # Update k for the next loop

    # For using the previous solution, use the following
    c_0_re = cs[0].real  
    c_0_im = cs[0].imag
    c_0 = c_0_re + 1j*c_0_im
    c_1 = c_0 + d_c

    print(f'k = {K[iter_outer]}, c = {cs[iter_outer]}')
print("--- %s seconds ---" % (time.time() - start_time))

0
k = 0.010000000000001, c = (0.49256739311450826+0.011822646467804189j)
--- 0.9161689281463623 seconds ---
